# Optymalizacja i najlepsze praktyki

**Cel szkoleniowy:** Opanowanie technik optymalizacji performance'u zapytań i tabel Delta w Databricks.

**Zakres tematyczny:**
- Optymalizacja zapytań: predicate pushdown, file pruning, column pruning
- Analiza planu fizycznego (explain())
- Optymalizacja tabel: partitioning, small files problem
- Auto optimize / auto compaction
- Dobór rozmiaru plików i strategii ZORDER
- Liquid Clustering - nowoczesna alternatywa dla partycjonowania

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Delta Lake & Lakehouse
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 16.4 LTS lub nowszy (zalecane: 17.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers lub **Serverless Compute** (zalecane)
- **Zależności**: Wykonany notebook `01_delta_lake_operations.ipynb`
- **Czas realizacji**: ~45 minut

> **Uwaga (2025):** Serverless Compute jest teraz domyślnym trybem dla nowych workloadów.

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie kluczowych mechanizmów optymalizacji w Databricks i Delta Lake.

**Typy optymalizacji:**

**1. Optymalizacja zapytań (Query Optimization):**
- **Predicate Pushdown**: Przeniesienie filtrów jak najniżej w planie wykonania
- **Column Pruning**: Odczyt tylko wymaganych kolumn (kolumnowy format Parquet)
- **File Pruning**: Ominięcie plików nieistotnych dla zapytania
- **Join Optimization**: Broadcast joins, bucket joins, sortmerge joins

**2. Optymalizacja tabel (Table Optimization):**
- **Partitioning**: Fizyczne rozdzielenie danych według kluczy
- **Z-Ordering**: Klasterowanie danych w plikach według wybranych kolumn
- **Compaction (OPTIMIZE)**: Łączenie małych plików w większe
- **Auto Compaction**: Automatyczne łączenie podczas zapisu

**3. Small Files Problem:**
Problem wydajności wywołany przez zbyt wiele małych plików w tabeli. Spark preferuje pliki 128MB-1GB dla optymalnej wydajności.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, count, avg, sum, max, min
from pyspark.sql.types import *
import time

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

**Kontekst użytkownika:**

In [ ]:
display(
    spark.createDataFrame([
        ("CATALOG", CATALOG),
        ("BRONZE_SCHEMA", BRONZE_SCHEMA),
        ("SILVER_SCHEMA", SILVER_SCHEMA),
        ("GOLD_SCHEMA", GOLD_SCHEMA),
        ("USER", raw_user)
    ], ["Variable", "Value"])
)

## Sekcja 0: Przygotowanie danych

Ten notebook jest **w pełni niezależny** - sam ładuje dane źródłowe i tworzy tabele potrzebne do demo optymalizacji.

In [ ]:
# Ścieżki do danych źródłowych
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

# Nazwy tabel (unikalne dla tego notebooka)
ORDERS_OPT = f"{BRONZE_SCHEMA}.orders_optimization"
CUSTOMERS_OPT = f"{BRONZE_SCHEMA}.customers_optimization"

**Wczytanie danych źródłowych:**

In [ ]:
# Wczytaj customers
customers_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(CUSTOMERS_CSV)

# Wczytaj orders
orders_df = spark.read.json(ORDERS_JSON)

# Dodaj kolumnę order_date (data bez czasu) dla partycjonowania
orders_df = orders_df.withColumn(
    "order_date", 
    F.to_date(F.col("order_datetime"))
)

print(f"✓ Customers: {customers_df.count()} rekordów")
print(f"✓ Orders: {orders_df.count()} rekordów")

**Zapisz jako tabele Delta do optymalizacji:**

In [ ]:
# Zapisz jako tabele Delta
customers_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(CUSTOMERS_OPT)

orders_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(ORDERS_OPT)

display(spark.createDataFrame([
    ("CUSTOMERS_OPT", CUSTOMERS_OPT, str(customers_df.count())),
    ("ORDERS_OPT", ORDERS_OPT, str(orders_df.count()))
], ["Tabela", "Full Name", "Rekordy"]))

## Sekcja 1: Analiza planu fizycznego (explain())

**Cel sekcji:** Naucz się analizować plany wykonania zapytań, aby zidentyfikować bottlenecki wydajnościowe.

**Teoria:**
Plan fizyczny to szczegółowa mapa tego, jak Spark wykonuje zapytanie:
- **Stages**: Logiczne etapy przetwarzania
- **Tasks**: Jednostki pracy wykonywane na partycjach
- **Shuffles**: Wymiana danych między executor'ami
- **Pushdowns**: Optymalizacje przeniesione do źródła danych

**Rodzaje explain():**
- `explain()` - podstawowy plan
- `explain(True)` - pełny plan z detalami
- `explain('extended')` - rozszerzony plan
- `explain('cost')` - plan z kosztem

### Przykład 1.1: Analiza planu prostego zapytania

In [0]:
# Przykład 1.1 - Analiza planu prostego zapytania

simple_query = spark.sql(f"""
    SELECT customer_id, first_name, last_name, customer_segment, city
    FROM {CUSTOMERS_OPT}
    WHERE customer_segment = 'Premium'
    ORDER BY customer_id DESC
    LIMIT 10
""")

**Podstawowy plan zapytania:**

In [ ]:
simple_query.explain()

**Rozszerzony plan zapytania (z detalami):**

In [ ]:
simple_query.explain(True)

### Przykład 1.2: Predicate Pushdown w praktyce

**Teoria:** Predicate pushdown to optymalizacja, gdzie filtry (warunki WHERE) są "przepychane" jak najniżej w planie wykonania, najlepiej do poziomu czytania plików. Dzięki temu czytamy tylko dane, które spełniają warunki.

In [0]:
# Przykład 1.2 - Predicate Pushdown

filtered_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_OPT}
    WHERE total_amount > 100 
    AND order_date >= '2024-01-01'
""")

**Sprawdź plan - poszukaj "PushedFilters" w planie:**

In [ ]:
# Sprawdź plan - poszukaj "PushedFilters" w planie
filtered_query.explain(True)

**💡 W planie szukaj:**
- `PushedFilters` - filtry przepchnięte do poziomu czytania
- `ReadSchema` - tylko wybrane kolumny (column pruning)  
- `PartitionFilters` - filtry na partycjach

## Sekcja 2: Strategia partycjonowania

**Cel sekcji:** Nauka wyboru optymalnych kluczy partycjonowania dla najlepszej wydajności.

**Teoria partycjonowania:**
- **Partitioning**: Fizyczne rozdzielenie tabeli na katalogi według wartości kolumn
- **Partition Pruning**: Spark pomija całe partycje, które nie są potrzebne dla zapytania
- **Idealne partycje**: 1-10GB danych na partycję, nie więcej niż 10,000 partycji

**Najlepsze praktyki:**
- Partycjonuj według kolumn często używanych w filtrach
- Unikaj partycjonowania według kolumn o wysokiej kardinalności
- Preferuj kolumny z naturalną hierarchią czasową (rok/miesiąc/dzień)
- Unikaj zbyt wielu małych partycji (small files problem)

### Przykład 2.1: Tworzenie tabeli partycjonowanej

In [0]:
# Przykład 2.1 - Tworzenie tabeli partycjonowanej

# Utwórz tabelę partycjonowaną według roku i miesiąca
ORDERS_PARTITIONED = f"{BRONZE_SCHEMA}.orders_opt_partitioned"

# Dodaj kolumny do partycjonowania
orders_with_partitions = spark.sql(f"""
    SELECT 
        *,
        YEAR(order_date) as year,
        MONTH(order_date) as month
    FROM {ORDERS_OPT}
""")

**Zapisz jako tabelę partycjonowaną:**

In [ ]:
orders_with_partitions.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .saveAsTable(ORDERS_PARTITIONED)

**Sprawdź strukturę partycji:**

In [ ]:
display(
    spark.sql(f"DESCRIBE DETAIL {ORDERS_PARTITIONED}")
    .select("name", "location", "partitionColumns")
)

### Przykład 2.2: Partition Pruning w działaniu

In [0]:
# Przykład 2.2 - Partition Pruning

# Zapytanie który wykorzystuje partycje (rok/miesiąc)
efficient_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_PARTITIONED}
    WHERE year = 2024 AND month = 1
""")

**Sprawdź plan - partition pruning w działaniu:**

In [ ]:
# Sprawdź plan - poszukaj "PartitionFilters"
efficient_query.explain(True)

**Porównanie: zapytanie BEZ partition pruning:**

In [ ]:
# Zapytanie które nie wykorzystuje partycji (nie filtruje po roku/miesiącu)
inefficient_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_PARTITIONED}
    WHERE customer_id = 1
""")

inefficient_query.explain(True)

## Sekcja 3: Small Files Problem

**Cel sekcji:** Zrozumienie i rozwiązanie problemu małych plików w Delta Lake.

**Co to jest Small Files Problem?**
- Gdy tabela ma zbyt wiele małych plików (< 128MB każdy)
- Spark preferuje pliki 128MB-1GB dla optymalnej wydajności
- Małe pliki powodują overhead w metadanych i zmniejszają throughput

**Przyczyny małych plików:**
- Częste zapisy INSERT w małych batch'ach
- Wysokie partycjonowanie z małą ilością danych na partycję
- Streaming z krótkimi trigger intervals

**Rozwiązania:**
- **OPTIMIZE** - łączy małe pliki w większe
- **Auto Compaction** - automatyczne łączenie podczas zapisu
- **Repartition** przed zapisem
- **Coalesce** dla zmniejszenia liczby partycji

### Przykład 3.1: Symulacja Small Files Problem

In [0]:
# Przykład 3.1 - Symulacja Small Files Problem

SMALL_FILES_TABLE = f"{BRONZE_SCHEMA}.small_files_demo"

**Symuluj wiele małych zapisów (każdy tworzy osobny plik):**

In [ ]:
for i in range(5):
    small_batch = spark.range(i*100, (i+1)*100).select(
        col("id"),
        (col("id") * 2).alias("value"),
        lit(f"batch_{i}").alias("batch_name")
    )
    
    small_batch.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(SMALL_FILES_TABLE)

**Sprawdź liczbę plików:**

In [ ]:
detail = spark.sql(f"DESCRIBE DETAIL {SMALL_FILES_TABLE}").collect()[0]

display(
    spark.createDataFrame([
        ("Liczba plików", str(detail['numFiles'])),
        ("Rozmiar tabeli", f"{detail['sizeInBytes']} bytes"),
        ("Średni rozmiar pliku", f"{detail['sizeInBytes'] / detail['numFiles']:.0f} bytes"),
        ("Status", "⚠️ Problem: zbyt wiele małych plików!")
    ], ["Metric", "Value"])
)

### Przykład 3.2: Rozwiązanie - OPTIMIZE i Auto Compaction

In [0]:
# Przykład 3.2 - Rozwiązanie Small Files Problem

# Wykonaj OPTIMIZE na tabeli z małymi plikami
spark.sql(f"OPTIMIZE {SMALL_FILES_TABLE}")

**Sprawdź stan po OPTIMIZE:**

In [ ]:
detail_after = spark.sql(f"DESCRIBE DETAIL {SMALL_FILES_TABLE}").collect()[0]

display(
    spark.createDataFrame([
        ("Liczba plików (PO OPTIMIZE)", str(detail_after['numFiles'])),
        ("Rozmiar tabeli", f"{detail_after['sizeInBytes']} bytes"),
        ("Średni rozmiar pliku", f"{detail_after['sizeInBytes'] / detail_after['numFiles']:.0f} bytes")
    ], ["Metric", "Value"])
)

**Włącz Auto Compaction dla przyszłych zapisów:**

In [ ]:
spark.sql(f"""
    ALTER TABLE {SMALL_FILES_TABLE}
    SET TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true'
    )
""")

**Sprawdź włączone właściwości Auto Compaction:**

In [ ]:
properties = spark.sql(f"SHOW TBLPROPERTIES {SMALL_FILES_TABLE}").collect()
auto_props = [(p['key'], p['value']) for p in properties if 'autoOptimize' in p['key']]

display(spark.createDataFrame(auto_props, ["Property", "Value"]))

### Przykład 3.3: VACUUM - Usuwanie starych plików

**Teoria:**
VACUUM usuwa stare pliki, które nie są już potrzebne (po operacjach DELETE, UPDATE, MERGE, OPTIMIZE). 
Domyślnie Delta Lake zachowuje pliki przez 7 dni (168 godzin) dla Time Travel.

**⚠️ Uwaga:** Po VACUUM nie można używać Time Travel do wersji wcześniejszych niż retention period!

In [ ]:
# Sprawdź ile plików można usunąć (DRY RUN)
spark.sql(f"VACUUM {SMALL_FILES_TABLE} DRY RUN")

**Wykonaj VACUUM (usuń stare pliki):**

> **Uwaga:** W produkcji używaj domyślnego retention (7 dni). Poniższy kod z `RETAIN 0 HOURS` jest tylko do demo!

In [ ]:
# Wykonaj VACUUM - usuń stare pliki
# W środowisku demo wyłączamy sprawdzenie retention
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

# VACUUM z krótkim retention (TYLKO DO DEMO!)
spark.sql("""
    VACUUM orders_opt RETAIN 1 HOURS
""")

# Przywróć domyślne ustawienie
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

print("✅ VACUUM wykonany - stare pliki usunięte")

## Sekcja 4: ZORDER BY - Advanced Clustering

**Cel sekcji:** Nauka wykorzystania ZORDER BY do optymalizacji zapytań z filtrami i joinami.

**Co to jest ZORDER BY?**
- Multi-dimensional clustering algorithm w Delta Lake
- Organizuje dane w plikach według wartości wybranych kolumn
- Poprawia data skipping - pomijanie niepotrzebnych plików podczas czytania
- Szczególnie skuteczny dla kolumn często używanych w filtrach WHERE i JOIN

**Kiedy używać ZORDER:**
- Kolumny często filtrowane w zapytaniach
- Kolumny używane w JOIN operations
- High-cardinality columns (wiele unikalnych wartości)
- Maksymalnie 3-4 kolumny (więcej = diminishing returns)

**ZORDER vs Partitioning:**
- Partitioning: fizyczne rozdzielenie na katalogi
- ZORDER: logiczne uporządkowanie w plikach (zachowuje pojedynczą strukturę folderów)

### Przykład 4.1: ZORDER BY dla często filtrowanych kolumn

In [0]:
# Wykonaj ZORDER BY na najczęściej filtrowanych kolumnach
spark.sql(f"""
    OPTIMIZE {ORDERS_OPT}
    ZORDER BY (customer_id, order_date)
""")

### Przykład 4.2: Pomiar skuteczności ZORDER - Data Skipping

In [0]:
# Przykład 4.2 - Pomiar skuteczności ZORDER

import time

# Zapytanie wykorzystujące kolumny z ZORDER
# customer_id to STRING (np. CUST000123), order_date to DATE
test_query = spark.sql(f"""
    SELECT COUNT(*) as cnt, AVG(total_amount) as avg_amount
    FROM {ORDERS_OPT}
    WHERE customer_id BETWEEN 'CUST000100' AND 'CUST000500'
    AND order_date >= '2024-06-01'
""")

**Pomiar czasu wykonania:**

In [ ]:
start_time = time.time()
result = test_query.collect()
elapsed = time.time() - start_time

display(
    spark.createDataFrame([
        ("Wynik", str(result[0])),
        ("Czas wykonania", f"{elapsed:.2f}s")
    ], ["Metric", "Value"])
)

**Plan zapytania - sprawdź data skipping:**

In [ ]:
# Sprawdź plan zapytania - data skipping
test_query.explain(True)

**💡 W planie szukaj:**
- `numFilesTotal` vs `numFilesSelected` - ile plików pominięto
- `metadata time` - czas parsowania metadanych
- `files pruned` - data skipping statistics

## Sekcja 5: Liquid Clustering - Przyszłość optymalizacji

**Cel sekcji:** Poznanie Liquid Clustering - nowoczesnej techniki zastępującej Hive Partitioning i ZORDER.

**Co to jest Liquid Clustering?**
To elastyczny mechanizm układania danych, który:
- Nie wymaga sztywnej struktury katalogów (jak Partitioning)
- Pozwala na zmianę kluczy klastrowania bez przepisywania całej tabeli
- Eliminuje problem "Small Files" związany z nadmiernym partycjonowaniem
- Działa inkrementalnie (nie trzeba optymalizować całej tabeli na raz)

**Kiedy używać?**
- Zamiast partycjonowania dla większości nowych tabel
- Gdy klucze partycjonowania mają wysoką kardynalność
- Gdy wzorce zapytań zmieniają się w czasie

In [ ]:
LIQUID_TABLE = f"{BRONZE_SCHEMA}.orders_opt_liquid"

# Tworzymy tabelę używając CLUSTER BY zamiast PARTITIONED BY
spark.sql(f"""
CREATE OR REPLACE TABLE {LIQUID_TABLE}
CLUSTER BY (customer_id, order_date)
AS SELECT * FROM {ORDERS_OPT}
""")

**Sprawdź właściwości tabeli:**

In [ ]:
# Sprawdź właściwości tabeli
display(spark.sql(f"DESCRIBE DETAIL {LIQUID_TABLE}").select("name", "clusteringColumns"))

### Przykład 5.2: Inkrementalna optymalizacja

**Teoria:**
W przeciwieństwie do ZORDER, który musi przeliczyć całą partycję/tabelę, Liquid Clustering działa inkrementalnie. `OPTIMIZE` uporządkuje tylko te dane, które tego wymagają (np. nowo dodane), co oszczędza czas i zasoby.

In [ ]:
# Uruchom OPTIMIZE - Liquid Clustering wie jak układać dane na podstawie definicji tabeli
spark.sql(f"OPTIMIZE {LIQUID_TABLE}")

**Sprawdź historię, aby zobaczyć operację CLUSTERING:**

In [ ]:
display(
    spark.sql(f"DESCRIBE HISTORY {LIQUID_TABLE}")
    .select("version", "operation", "operationParameters")
    .limit(5)
)

### Porównanie: Liquid Clustering vs Partitioning + ZORDER

| Cecha | Partitioning + ZORDER | Liquid Clustering |
|-------|-----------------------|-------------------|
| **Konfiguracja** | Wymaga starannego doboru kolumn partycjonowania | Elastyczne `CLUSTER BY` |
| **Small Files** | Ryzyko przy nadmiernym partycjonowaniu | Automatycznie zarządzane |
| **Zmiana klucza** | Trudna (wymaga przepisania tabeli) | Łatwa (`ALTER TABLE CLUSTER BY`) |
| **Optymalizacja** | `OPTIMIZE ZORDER BY` (kosztowne) | `OPTIMIZE` (inkrementalne) |
| **Skew Data** | Podatne na data skew | Odporne na data skew |

**Rekomendacja:** Używaj Liquid Clustering dla wszystkich nowych tabel w Databricks Runtime 13.3+, chyba że masz specyficzny powód, by używać partycjonowania (np. kompatybilność ze starszymi czytnikami).

## Best Practices - Przewodnik optymalizacji

### 🎯 Strategia optymalizacji (w kolejności priorytetów):

**1. Analiza workload:**
- Zidentyfikuj najczęściej wykonywane zapytania
- Znajdź kolumny najczęściej używane w filtrach WHERE
- Sprawdź które zapytania zajmują najwięcej czasu

**2. Optymalizacja zapytań:**
- Używaj filtrów WHERE jak najwcześniej w zapytaniu
- Wybieraj tylko potrzebne kolumny (SELECT specific columns, nie *)
- Preferuj JOIN na zindeksowanych kolumnach
- Używaj LIMIT gdy możliwe

**3. Optymalizacja tabel:**
- **Partitioning**: Tylko dla dużych tabel (>1TB) i często filtrowanych kolumn
- **ZORDER BY**: Dla 2-4 najczęściej filtrowanych kolumn
- **OPTIMIZE**: Regularnie (np. daily) dla aktywnych tabel
- **Auto Compaction**: Włącz dla tabel z częstymi zapisami

**4. Monitoring i maintenance:**
- Regularnie sprawdzaj `DESCRIBE DETAIL` - liczba plików, rozmiar
- Uruchamiaj `VACUUM` co najmniej raz w tygodniu
- Monitoruj Spark UI dla długo działających zapytań
- Używaj `explain()` do analizy problemowych zapytań

## Troubleshooting - Diagnoza problemów wydajnościowych

### 🔍 Najczęstsze problemy i rozwiązania:

**Problem 1: Zapytanie działa bardzo wolno**
```python
# Diagnoza:
df.explain(True)  # Sprawdź plan wykonania
```
**Możliwe przyczyny:**
- Brak filtrów - czyta całą tabelę
- Shuffle operations - dużo ruchu sieciowego  
- Skewed data - nierównomierne rozłożenie danych
- Small files - zbyt wiele małych plików

**Problem 2: "OutOfMemoryError" podczas JOIN**
**Rozwiązanie:**
```python
# Zwiększ partycje przed JOIN
df1 = df1.repartition(200, "join_key")
df2 = df2.repartition(200, "join_key")

# Lub użyj broadcast join dla małych tabel
from pyspark.sql.functions import broadcast
result = large_df.join(broadcast(small_df), "key")
```

**Problem 3: Długie czasy zapisu do Delta**
**Rozwiązanie:**
- Włącz Auto Compaction
- Użyj `coalesce()` przed zapisem
- Avoid zbyt wysokie partycjonowanie

**Problem 4: OPTIMIZE nie poprawia wydajności**
**Przyczyna:** ZORDER BY jest potrzebny dla specific query patterns
```python
# Zamiast samego OPTIMIZE:
OPTIMIZE table_name

# Użyj OPTIMIZE z ZORDER:
OPTIMIZE table_name ZORDER BY (frequently_filtered_columns)
```

## Podsumowanie

### Co osiągnęliśmy:

- **Analiza wydajności**: Czytanie i interpretacja planów fizycznych z `explain()`
- **Predicate Pushdown**: Identyfikacja bottlenecków i pushed filters
- **Partycjonowanie**: Strategia partycjonowania według często filtrowanych kolumn
- **ZORDER BY**: Multi-dimensional clustering dla 2-4 kolumn
- **Small Files Problem**: Rozwiązywanie przez OPTIMIZE i Auto Compaction
- **Liquid Clustering**: Nowoczesna alternatywa dla partycjonowania

### Kluczowe wnioski:

| # | Zasada |
|---|--------|
| 1 | **Analiza przed optymalizacją** - zawsze najpierw `explain()` |
| 2 | **Partitioning ≠ ZORDER** - różne techniki dla różnych przypadków |
| 3 | **Small files = performance killer** - regularne OPTIMIZE |
| 4 | **ZORDER BY** - maksymalnie 3-4 kolumny, wybieraj najczęściej filtrowane |
| 5 | **Liquid Clustering** - preferuj dla nowych tabel w DBR 13.3+ |

### Metryki do monitorowania:

| Metryka | Dobra wartość | Akcja jeśli przekroczona |
|---------|---------------|--------------------------|
| Liczba plików | < 1000/TB | OPTIMIZE |
| Średni rozmiar pliku | 128MB-1GB | OPTIMIZE + Auto Compaction |
| Skipped files ratio | >80% | ZORDER BY |

### Następne kroki:

📚 **Kolejny dzień:** DZIEN_3 - Transformacje i Governance

## Cleanup

Opcjonalnie usuń tabele demo utworzone podczas ćwiczeń:

In [0]:
# Cleanup - usuń tabele demo utworzone w tym notebooku

# Odkomentuj poniższe linie aby usunąć tabele demo:

# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_OPT}")
# spark.sql(f"DROP TABLE IF EXISTS {CUSTOMERS_OPT}")
# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_PARTITIONED}")
# spark.sql(f"DROP TABLE IF EXISTS {SMALL_FILES_TABLE}")
# spark.sql(f"DROP TABLE IF EXISTS {LIQUID_TABLE}")

# print("✅ Wszystkie tabele demo usunięte")

print("ℹ️ Cleanup wyłączony (odkomentuj kod aby usunąć tabele demo)")